<a href="https://colab.research.google.com/github/o8ma19/1st-DL-CVMarathon/blob/master/homework/Day021_Transfer_Learning_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [8]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''
model=keras.applications.xception.Xception(include_top=False, weights='imagenet',\
                                  input_tensor=input_tensor,pooling=None, classes=10)

'''Resnet 50 架構'''
#model=keras.applications.ResNet50(include_top=False, weights='imagenet',
                                    #input_tensor=input_tensor,
                                    #pooling=None, classes=10)
model.summary()


83689472/83683744 [==============================] - 2s 0us/step
Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 15, 15, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 15, 15, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 15, 15, 32)   0           block1_conv1_bn[0][0]            
__________________________

## 添加層數

In [9]:
x = model.output

'''可以參考Cifar10實作章節'''
x = GlobalAveragePooling2D()(x)
x = Dense(output_dim=128, activation='relu')(x)
x=Dropout(p=0.1)(x)

predictions = Dense(output_dim=10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))

Model深度： 136


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  


## 鎖定特定幾層不要更新權重

In [0]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [11]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

(50000, 32, 32, 3)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## Training

In [7]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=20)

Epoch 1/20
50000/50000 [==============================] - 69s 1ms/step - loss: 0.6663 - acc: 0.7687
Epoch 2/20
50000/50000 [==============================] - 64s 1ms/step - loss: 0.6045 - acc: 0.7971
Epoch 3/20
50000/50000 [==============================] - 63s 1ms/step - loss: 0.5797 - acc: 0.8082
Epoch 4/20
50000/50000 [==============================] - 63s 1ms/step - loss: 0.5173 - acc: 0.8205
Epoch 5/20
50000/50000 [==============================] - 63s 1ms/step - loss: 0.4304 - acc: 0.8555
Epoch 6/20
50000/50000 [==============================] - 63s 1ms/step - loss: 0.4258 - acc: 0.8537
Epoch 7/20
50000/50000 [==============================] - 63s 1ms/step - loss: 0.3889 - acc: 0.8650
Epoch 8/20
50000/50000 [==============================] - 63s 1ms/step - loss: 0.3614 - acc: 0.8740
Epoch 9/20
50000/50000 [==============================] - 63s 1ms/step - loss: 0.3405 - acc: 0.8776
Epoch 10/20
50000/50000 [==============================] - 63s 1ms/step - loss: 0.3101 - acc: 0.8926

In [12]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=20)

Epoch 1/20
50000/50000 [==============================] - 41s 814us/step - loss: 1.3684 - acc: 0.5180
Epoch 2/20
50000/50000 [==============================] - 36s 730us/step - loss: 1.0208 - acc: 0.6508
Epoch 3/20
50000/50000 [==============================] - 36s 729us/step - loss: 0.9032 - acc: 0.6901
Epoch 4/20
50000/50000 [==============================] - 36s 728us/step - loss: 0.8119 - acc: 0.7216
Epoch 5/20
50000/50000 [==============================] - 36s 730us/step - loss: 0.7490 - acc: 0.7412
Epoch 6/20
50000/50000 [==============================] - 36s 726us/step - loss: 0.6786 - acc: 0.7675
Epoch 7/20
50000/50000 [==============================] - 36s 728us/step - loss: 0.6117 - acc: 0.7886
Epoch 8/20
50000/50000 [==============================] - 37s 734us/step - loss: 0.5534 - acc: 0.8096
Epoch 9/20
50000/50000 [==============================] - 36s 727us/step - loss: 0.4977 - acc: 0.8283
Epoch 10/20
50000/50000 [==============================] - 36s 724us/step - loss: 